## Fidelity Dividend Tool

### Setup
1. Download transaction and activities as a `.csv` file from Fidelity. Recommended filtering for only Dividend Events
2. Move to the `Data/CSV/Fidelity/Dividends` folder
3. Make sure the downloaded csv file ends in `*-Dividends.csv` i.e. `2024-Dividends.csv`

### Final Format
|       Date |              Action | Symbol | Provider |    Account | Amount |
|-----------:|--------------------:|-------:|---------:|-----------:|-------:|
| 05/30/2024 |   Dividend Received |    QQM | Fidelity |      Index |   2.00 |
| 05/30/2024 | Dividend Reinvested |    VOO |  Merrill |   CMA-Edge |   4.35 |
| 05/30/2024 |   Dividend Received |    VTI | Fidelity | Individual |   23.5 |

In [ ]:
import os
import pandas as pd

# Configure width of display for pandas data frame
pd.set_option('display.width', 400)
# Configure maximum columns shown
pd.set_option('display.max_columns', 15)
# Do not limit number of rows displayed, allows scroll
pd.set_option("display.max_rows", None)

# Create a folder called Data at the root of this project
base_path = '../Data'
fidelity_col_converters = {'Run Date': pd.to_datetime,
                     'Account': str,
                     'Action': str,
                     'Symbol': str,
                     'Description': str,
                     'Type': str,
                     'Amount': float
                     }

# Constants
DIVIDEND_RECEIVED = 'Dividend Received'
DIVIDEND_REINVESTMENT = 'Dividend Reinvestment'
ACTION_COL = 'Action'
AMOUNT_COL = 'Amount'
ACCOUNT_COL = 'Account'
SYMBOL_COL = 'Symbol'
PROVIDER_COL = 'Provider'

## Fidelity CSV PreProcessing Handler

In [ ]:
def preprocess_fidelity(main_data: pd.DataFrame, fidelity_file_path: str) -> pd.DataFrame:
    # Read in csv data with the inferred data type and specified columns
    df = pd.read_csv(file_path, converters=fidelity_col_converters, usecols=lambda cols: cols in fidelity_col_converters.keys())
    
    # Rename "Run Date" col to "Date
    df.rename(columns={"Run Date": "Date",}, inplace=True)
    
    # Drop unneeded columns
    df.drop(columns=['Description', 'Type'], inplace=True)
    
    # Remove the Account number - .str is needed because it's working with a Series data type
    df['Account'] = df['Account'].str.rsplit(' ', n=1).str[0]
    
    # Remap Action to only Dividend Received or Dividend Reinvestment. Special cases will map based on pos or neg amt
    df['Action'] = df.apply(lambda row:  
    DIVIDEND_RECEIVED if row[ACTION_COL].strip().startswith('DIVIDEND RECEIVED') else 
    DIVIDEND_REINVESTMENT if row[ACTION_COL].strip().startswith('REINVESTMENT')  else 
    DIVIDEND_RECEIVED if row[AMOUNT_COL] >= 0 else DIVIDEND_REINVESTMENT,
                            axis=1)
    
    # Map Missing Symbol (i.e. Capital One rows) if missing - cell for C1 shows as empty string, not null
    account_to_symbol_map = {
        "CAPITAL ONE 401K ASP": "COF"
    }
    df['Symbol'] = df.apply(lambda row:
                            account_to_symbol_map.get(row[ACCOUNT_COL].strip(), 'Cannot find symbol') if pd.isna(row[SYMBOL_COL]) 
                                                                                                         or pd.isnull(row[SYMBOL_COL]) or row[SYMBOL_COL] is None or row[SYMBOL_COL] == '' else row[SYMBOL_COL], axis=1
                            )
    # Add Provider Column to all rows
    df[PROVIDER_COL] = "Fidelity"
    
    # Re-arrange columns
    df = df[['Date', 'Provider', 'Account', 'Action', 'Symbol', 'Amount']]
    
    # Append to data
    print(f"Fidelity|{file_path[3:]}")
    return pd.concat([main_data, df], ignore_index=True)

## Merrill CSV PreProcessing Handler

## Read in CSV Files
Read in the CSV files under `Data/Fidelity/Dividends` to combine into a single data frame for further processing.
Clean up by removing the account no.

In [ ]:
# Main Data frame, empty by default
data = pd.DataFrame()

for root, dirs, files in os.walk(base_path):
    for file in files:
        if 'Dividends' in root and '-Dividends.csv' in file:
            file_path = os.path.join(root, file)
            
            if "Fidelity" in file_path:
                data = preprocess_fidelity(data, file_path)
            
            
            
            
print(data)

## Dividends Received Grouped By Run Date + Symbol
Filter for only Dividends received and combine the amounts if matching date and symbol

In [ ]:
dividends_data = pd.DataFrame(data)
dividends_received_by_symbol = dividends_data[dividends_data['Action'].str.contains('DIVIDEND RECEIVED', case=False, na=False)]

dividends_received = (dividends_received_by_symbol
                      .groupby(['Run Date', 'Symbol'], as_index=False)
                      .sum())

print(dividends_received[['Run Date', 'Symbol', 'Amount']])

## Dividends Received Grouped By Run Date (Year-Month) + Account
Filter for only Dividends received and combine the amounts if matching date and symbol

In [ ]:
dividends_data = pd.DataFrame(data)
# Copy was explicitly used to avoid the warning when adding the ''Year-Month' that it would affect the original
dividends_received_by_account = dividends_data[dividends_data['Action'].str.contains('DIVIDEND RECEIVED', case=False, na=False)].copy()

dividends_received_by_account.loc[:,'Year-Month'] = dividends_received_by_account['Run Date'].dt.to_period('M')

dividends_received_by_account = (dividends_received_by_account
                                 .groupby(['Year-Month', 'Account'], as_index=False)['Amount']
                                 .sum())
print(dividends_received_by_account[['Year-Month', 'Account', 'Amount']])
